In [21]:
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader("data.txt")
text_document = text_loader.load()
text_loader

In [22]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document = text_splitter.split_documents(text_document)

In [23]:
from dotenv import load_dotenv
import os
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(document, embeddings)

In [24]:
query = "How do I reset my password?"
docs = db.similarity_search(query)
docs[0].page_content

'1. FAQs\n\nQ: How can I reset my password?\nA: Go to Account Settings > Security > Reset Password. Youâ€™ll receive an email link to set a new one.\n\nQ: What payment methods do you accept?\nA: We accept Credit/Debit Cards, Net Banking, UPI, and Wallets (Paytm, PhonePe).\n\nQ: Can I cancel an order after placing it?\nA: Yes, you can cancel an order within 24 hours before it is shipped.\n\n2. Help Docs (Short Guides)\n\nPassword Reset Guide:\n\nClick Forgot Password on the login page.\n\nEnter your registered email/phone.\n\nFollow the reset link sent to your email.\n\nTracking Orders:\n\nGo to My Orders section.\n\nSelect the order you want to track.\n\nYouâ€™ll see live tracking status (Processing â†’ Shipped â†’ Out for Delivery â†’ Delivered).\n\n3. Product Guide (E-commerce Example)\n\nReturn Policy for Electronics:\n\nElectronics can be returned within 7 days if unopened.\n\nOpened products can only be replaced if defective.\n\nWarranty claims must go through the manufacturer.\n\

In [27]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = OpenAI()

prompt = ChatPromptTemplate.from_template(
    "You are a helpful customer support assistant. Use the following pieces of context to answer the question at the end.\n"
    "context :{context} \n\n"
    "Question: {input}\n"
)

from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, documents_chain)

response = retrieval_chain.invoke({"input": query})

response['answer']

'Answer: To reset your password, go to Account Settings > Security > Reset Password. You will receive an email with a link to set a new password.'